# Sample Skyspark VAV Validation

# 1) Setup

## Imports

In [1]:
# ----------------------------------------
# Imports
# ----------------------------------------
import os
import json
import re

from rdflib import Namespace, SH, RDF, BNode, Graph
from pyshacl import validate

from tasty import constants as tc
from tasty import graphs as tg
from tasty.skyspark import api_request as api
from tasty.skyspark import process_graphs as pg

## Inputs
Define the key variables and input information here

***Items to Change***
- `SHAPE`: this is the name of the SHACL equipment shape against which you would like to validate your sample equipment in the instance data
- `SAMPLE`: this is the name of the sample equipment in your instance data
- `input_namespace_uri`: this is the namespace uri used for your sample equipment in the instance data
- `raw_data_graph_filename`: this is the filename/filepath to save the raw instance data (in turtle format) retrieved from the Skyspark API call
- `data_graph_filename`: this is the filename/filepath to save the cleaned/processed instance data for the data graph to be used for validation
- `shapes_graph_filename`: this it the filename/filepath of the SHACL shapes data for the shape graph 
***Remaining Items*** </br>
These items should be okay as is, but can be changed if need be. If you are printing out results, <u>*make sure that the output directory exists in your local file structure*</u>.
- `output_directory`: this is the directory where output files will be printed to below
- `tasty_main_directory`: this is the absolute path of the main tasty directory. It should just be the parent directory of the current working directory.

In [2]:
# ----------------------------------------
# User Defined Variables
# ----------------------------------------
SHAPE = 'NREL-VAV-SD-Cooling-Only-Shape'
SAMPLE = '214466de-7abb28a7'
input_namespace_uri = 'urn:/_#'

raw_data_graph_filename = 'tests/files/data/sample_skyspark_vav_raw.ttl'
data_graph_filename = 'tests/files/data/sample_skyspark_vav_clean.ttl'
shapes_graph_filename = 'tasty/generated_shapes/haystack_all.ttl'

output_directory = os.path.join(os.path.abspath(''), 'example_data/output')
tasty_main_directory = os.path.join(os.path.abspath(''), '../')
# print(tasty_main_directory)

# ----------------------------------------
# Variables and Constants
# ----------------------------------------

NAMESPACE = Namespace(input_namespace_uri)
shape_name = tc.PH_SHAPES_NREL[SHAPE]
target_node = NAMESPACE[SAMPLE]

## API Request From Skyspark 
NOTE - Must be connected to NREL network to access the api endpoint

In [3]:
axon_query_string = '(point and equipRef->navName=="UFVAV-3") or (equip and navName=="UFVAV-3")'
api_url_endpoint = 'https://internal-apis.nrel.gov/intelligentcampus/stm_campus/read'

response = api.make_get_request(api_url_endpoint, axon_query_string, 'turtle')

print(response.status_code, end = " - ")
if response.status_code == 200:
    print("Sucess")
elif response.status_code == 404:
    print("Not Found")

raw_skyspark_data = response.text
# print(raw_skyspark_data)

200 - Sucess


In [4]:
# Save response to file
f = os.path.join(tasty_main_directory, raw_data_graph_filename)

api.save_response(raw_skyspark_data, f)

print(f"raw instance data saved to '{raw_data_graph_filename}' ")

raw instance data saved to 'tests/files/data/sample_skyspark_vav_raw.ttl' 


### Pre-Process Raw Input File

In [5]:
# ----------------------------------------
# Pre Process raw skyspark .ttl file 
# ----------------------------------------
f1 = os.path.join(tasty_main_directory, raw_data_graph_filename)
f2 = os.path.join(tasty_main_directory, data_graph_filename)

pg.clean_raw_skyspark_turtle(f1,f2)

print(f"cleaned instance data saved to '{data_graph_filename}' ")

cleaned instance data saved to 'tests/files/data/sample_skyspark_vav_clean.ttl' 


# 2) Generate Graphs

### Create Data, Shapes, and Ontology Graphs

In [6]:
# ----------------------------------------
# Generate Graphs
# ----------------------------------------

schema = tc.HAYSTACK
version = tc.V3_9_10

# Data Graph
dg_file = os.path.join(tasty_main_directory, data_graph_filename)
data_graph = pg.get_data_graph(dg_file, schema, version)
print("...loaded data graph")

# Shapes Graph
sg_file = os.path.join(tasty_main_directory, shapes_graph_filename)
shapes_graph = pg.get_shapes_graph(sg_file, target_node, shape_name, schema, version)
print("...loaded shapes graph")

# Ontology Graph
ont_graph = pg.get_ontology_graph(schema, version)
print("...loaded ontology graph")

...generated tag list
...adding namespaces
...processing node: 	urn:/_#211c90b7-f2430aa1
...processing node: 	urn:/_#211c90b6-0da18cd7
...processing node: 	urn:/_#22f5d821-8e310c3e
...processing node: 	urn:/_#211c90ab-701fa511
...processing node: 	urn:/_#211c90b6-52117d4d
Point: 	urn:/_#211c90b7-f2430aa1
Tags: 	sensor
	unocc
	...First Class Entity Type: point

Point: 	urn:/_#211c90b6-0da18cd7
Tags: 	damper
	discharge
	air
	cmd
	...First Class Entity Type: point

Point: 	urn:/_#22f5d821-8e310c3e
Tags: 	sensor
	occupied
	...First Class Entity Type: point

Point: 	urn:/_#211c90ab-701fa511
Tags: 	sensor
	zone
	air
	temp
	...First Class Entity Type: air-temp-sensor

Point: 	urn:/_#211c90b6-52117d4d
Tags: 	temp
	zone
	air
	sp
	...First Class Entity Type: air-temp-sp

...loaded data graph
...loaded shapes graph
...loaded ontology graph


In [7]:
pg.print_graph(data_graph)

@prefix core: <https://skyfoundry.com/def/core/3.0.27#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix p_: <urn:/_#> .
@prefix ph: <https://project-haystack.org/def/ph/3.9.10#> .
@prefix phIoT: <https://project-haystack.org/def/phIoT/3.9.10#> .
@prefix phScience: <https://project-haystack.org/def/phScience/3.9.10#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ph:hasTag a owl:ObjectProperty ;
    rdfs:range ph:marker .

p_:211c90ab-701fa511 a phIoT:air-temp-sensor,
        phIoT:his-point ;
    rdfs:label "S&TF UFVAV-3 Zone Temp" ;
    ph:hasTag phIoT:zone ;
    ph:kind "Number" ;
    ph:tz "Denver" ;
    ph:unit "°F" ;
    phIoT:curStatus "unknown" ;
    phIoT:equipRef p_:214466de-7abb28a7 ;
    phIoT:hisStatus "ok" ;
    phIoT:siteRef p_:211601cb-e3e5e9b3 ;
    phIoT:spaceRef p_:2184f1be-29c172a8 ;
    core:disMacro "$equipRef $navName" ;
    core:navName "Zone Temp" ;
    p_:bacnetConnRef p_:211c8a83-ef5c639d

# 3) Validation

## PySHACL Validation

In [8]:
# ----------------------------------------
# Run pySCHACL Validation
# ----------------------------------------
result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=ont_graph)
conforms, results_graph, results = result

print(f"Conforms: {conforms}")

Conforms: False


In [9]:
pg.print_graph(results_graph)

@prefix nrel: <https://project-haystack.org/datashapes/nrel#> .
@prefix phIoT: <https://project-haystack.org/def/phIoT/3.9.10#> .
@prefix phShapes: <https://project-haystack.org/datashapes/core#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode <urn:/_#214466de-7abb28a7> ;
            sh:resultMessage "Value does not conform to every Shape in ('nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape', 'nrel:ZoneThermalComfortSensorsShape')" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeConstraintComponent ;
            sh:sourceShape nrel:NREL-VAV-SD-Cooling-Only-Sh

## Determine Missing Points
This implements a rudimentary logic for finding the missing points (simple shapes) from the pySHACL results graph. The process is as follows:
1. Find each "validation result" which represents one SHACL constraint that was not met. This is done by iterating through all the triples in the graph and finding the triple with a `rdf:type` of `sh:ValidationResult`. The subject of this match will be the URI of the "validation result" node.
2. For each of these "validation results" look at the `sh:sourceShape`
3. If it is a BNode (as opposed to a URI) then we assume this refers to one of the constraints on the functional group SHACL shape (and therefore one of the "simple shapes") and it will have a `sh:qualifiedValueShape` which should be a URI of one of the simple shapes.
4. Add this shape to the list of missing points

*Note: this logic likely needs to be refined*

In [7]:
missing_points = []

# Find the Validation Results
for subject, predicate, object in results_graph.triples((None, RDF.type, SH.ValidationResult)):
#     print(f"Subject:{subject}\tPredicate:{predicate}\tObject:{object}")

    # check if Validation result points to a BNode
    for node in results_graph.objects(subject=subject, predicate=SH.sourceShape):
#         print(f"\tNode:{node}\t\tIs BNode:{isinstance(node, BNode)}")

        if isinstance(node, BNode):
            point = results_graph.value(subject=node, predicate=SH.qualifiedValueShape)
            missing_points.append(point)

if len(missing_points) <= 0:
    print("No Points Missing")
else:
    print(f"{len(missing_points)} Missing Points:")
    for point in missing_points:
#         for subject, predicate, object in shapes_graph.triples((point, SH.class, None)):
#             print(f"Subject:{subject}\tPredicate:{predicate}\tObject:{object}")
        print(f"\t{point}")
            

26 Missing Points:
	https://project-haystack.org/datashapes/nrel#ZoneRelativeHumidityShape
	https://project-haystack.org/datashapes/nrel#ZoneTemperatureHeatingOccupiedSetpointShape
	https://project-haystack.org/datashapes/nrel#ZoneTemperatureHeatingUnoccupiedSetpointShape
	https://project-haystack.org/datashapes/nrel#ZoneTemperatureHeatingStandbySetpointShape
	https://project-haystack.org/datashapes/nrel#ZoneTemperatureHeatingEffectiveSetpointShape
	https://project-haystack.org/datashapes/nrel#MaximumCoolingDischargeAirFlowSetpointShape
	https://project-haystack.org/datashapes/nrel#MaximumHeatingDischargeAirFlowSetpointShape
	https://project-haystack.org/datashapes/nrel#DischargeAirDamperFeedbackShape
	https://project-haystack.org/datashapes/nrel#MinimumHeatingDischargeAirFlowSetpointShape
	https://project-haystack.org/datashapes/nrel#MinimumCoolingDischargeAirFlowSetpointShape
	https://project-haystack.org/datashapes/nrel#DischargeAirFlowSetpointShape
	https://project-haystack.org/dat